# 📒 train_risk_model.ipynb (Notebook Template - Beraborrow Risk Model w/ Blockscout)


## 📌 Objective
Train a risk classification model (0 = safe, 1 = risky) for Beraborrow vaults using on-chain activity data from Blockscout.

## 📦 Setup
!pip install pandas requests scikit-learn joblib




In [ ]:

## 🔧 Configuration

import requests
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

BLOCKSCOUT_API = "https://blockscout.berachain.dev/api"
BERA_CONTRACT = "0x2A63e2b30e5169486C1c1b4356e1Ec87Be1628bD"  # LSP contract as example



In [ ]:

## 📊 Fetch Transaction Data
def fetch_txns(contract):
    url = f"{BLOCKSCOUT_API}?module=account&action=txlist&address={contract}&sort=asc"
    r = requests.get(url)
    return pd.DataFrame(r.json()["result"])

transactions = fetch_txns(BERA_CONTRACT)
transactions['timeStamp'] = pd.to_datetime(transactions['timeStamp'], unit='s')
transactions['value_eth'] = transactions['value'].astype(float) / 1e18


In [ ]:
## 📊 Fetch Internal Transactions

def fetch_internal_txns(contract):
    url = f"{BLOCKSCOUT_API}?module=account&action=txlistinternal&address={contract}"
    r = requests.get(url)
    return pd.DataFrame(r.json()["result"])

internal_txns = fetch_internal_txns(BERA_CONTRACT)


In [ ]:
## 🧠 Feature Engineering

features = {
    "total_txns": len(transactions),
    "unique_users": transactions['from'].nunique(),
    "avg_value": transactions['value_eth'].mean(),
    "last_activity_days": (pd.Timestamp.now() - transactions['timeStamp'].max()).days,
    "internal_txn_count": len(internal_txns),
    "internal_gas_used_avg": internal_txns['gasUsed'].astype(float).mean() if len(internal_txns) > 0 else 0.0,
}


In [ ]:
## 🏷️ Labeling (Manual or Rule-Based for MVP)

features["label"] = 1 if features["last_activity_days"] > 14 else 0  # Example logic


In [ ]:
## 🧪 Build Dataset

df = pd.DataFrame([features])  # Append for multiple vaults if available

